In [ ]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from glob import glob
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cd drive/MyDrive/CRUPA; ls

ANTON		   colab_MISHA.ipynb  MISHA_heap    NIKITA.ipynb
ANTON_heap	   hard		      MISHA_videos  TimeDistributed.ipynb
ANTON_videos	   main		      model_save    Untitled0.ipynb
colab_ANTON.ipynb  MISHA	      NIKITA


In [ ]:
main_dir = 'drive/MyDrive/CRUPA/MISHA_heap_v2/'
main_dir_Anton = 'drive/MyDrive/CRUPA/ANTON_heap/'
main_dir_Mish = 'drive/MyDrive/CRUPA/MISHA_heap_v2/'

In [ ]:

img_shape = (224, 224)
images_mish = sorted(os.listdir(main_dir))
images_anton = sorted(os.listdir(main_dir_Anton))
#images_misha = os.listdir(main_dir_Mish)
images = images_mish + images_anton 

images_info_df = pd.DataFrame(columns=['image_name', 'mass', 'food_type'])
for ind, img in enumerate(images):
    items = img.split('_')
    file = img.split('_')[2]
    #print(file[0:2])
    if file[0:2] == 'mv':
      images_info_df.loc[ind] = [main_dir + img, int(items[0]), items[1]]
    elif file[0:2] == 'av':
      images_info_df.loc[ind] = [main_dir_Anton + img, int(items[0]), items[1]]
    
#images_info_df['image_name'] = sorted(images_info_df['image_name'])
mass_list = list(images_info_df['mass'])  
mass_list = np.array(mass_list)


In [ ]:
images_info_df[0:10000]

,image_name,mass,food_type
0,drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_...,100,grechka
1,drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_...,100,grechka
2,drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_...,100,grechka
3,drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_...,100,grechka
4,drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_...,100,grechka
...,...,...,...
1365,drive/MyDrive/CRUPA/ANTON_heap/90_grechka_av2_...,90,grechka
1366,drive/MyDrive/CRUPA/ANTON_heap/90_grechka_av2_...,90,grechka
1367,drive/MyDrive/CRUPA/ANTON_heap/90_grechka_av2_...,90,grechka
1368,drive/MyDrive/CRUPA/ANTON_heap/90_grechka_av2_...,90,grechka


In [ ]:
# images_in_matrix_form = []  # массив с картинками в матричном виде
# for img_name in images_info_df['image_name']:
#     img = cv2.imread(img_name)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, img_shape)
#     images_in_matrix_form.append(np.asarray(img))
# images_in_matrix_form = np.array(images_in_matrix_form) 

GENERATOR DATA

In [ ]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    #img = cv2.imread(file)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = cv2.resize(img, img_shape)
    images_batch.append(images_info_df['image_name'][i])
    mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    #print(images_batch)
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []


images_list = np.array(images_list)
mass_list = np.array(mass_list)    
print(images_list)
print(mass_list)

[['drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver0_frame_0.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver0_frame_1.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver0_frame_2.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver0_frame_3.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver0_frame_4.jpg']
 ['drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver1_frame_0.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver1_frame_1.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver1_frame_2.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver1_frame_3.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver1_frame_4.jpg']
 ['drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver2_frame_0.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver2_frame_1.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver2_frame_2.jpg'
  'drive/MyDrive/CRUPA/MISHA_heap_v2/100_grechka_mv1_ver2_frame_3.jpg'
  'd

In [ ]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_shape)
    images_batch.append(img)
    #mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []

images_list = np.array(images_list)
mass_list = np.array(mass_list)    
# print(images_list)
print(mass_list)

[100 100 100 100 100 100 100 100 100 100  10  10  10  10  10  10  10  10
  10  10 110 110 110 110 110 110 110 110 110 110 130 130 130 130 130 130
 130 130 130 130 140 140 140 140 140 140 140 140 140 140 150 150 150 150
 150 150 150 150 150 150 160 160 160 160 160 160 160 160 160 160 170 170
 170 170 170 170 170 170 170 170 180 180 180 180 180 180 180 180 180 180
 190 190 190 190 190 190 190 190 190 190 200 200 200 200 200 200 200 200
 200 200  20  20  20  20  20  20  20  20  20  20  30  30  30  30  30  30
  30  30  30  30  40  40  40  40  40  40  40  40  40  40  50  50  50  50
  50  50  50  50  50  50  60  60  60  60  60  60  60  60  60  60  70  70
  70  70  70  70  70  70  70  70  80  80  80  80  80  80  80  80  80  80
  90  90  90  90  90  90  90  90  90  90 100 100  10  10 110 110 120 120
 130 130 140 140 150 150 160 160 170 170 180 180 190 190 200 200  20  30
  30  40  40  50  50  60  60  70  70  80  90  90]


In [ ]:
print(len(mass_list))

228


MODEL

In [ ]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16

vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

for layer in vgg.layers[:-4]:
    layer.trainable = False

model = Sequential()

model.add(
    TimeDistributed(vgg, input_shape=(5, 224, 224, 3))
)

model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='linear'))
model.compile(optimizer = 'adam', loss='mean_absolute_error')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 5, 7, 7, 512)     14714688  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 5, 25088)         0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 256)               25953280  
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [ ]:
epochs = 20
batch_size = 64

In [ ]:
path = os.path.join('drive/MyDrive/CRUPA/model_save/', f'20210906_1623_v1_e{epochs}_b{batch_size}/')

if not os.path.exists(path):
    os.makedirs(path)

earlystopper = EarlyStopping(patience=15, verbose=1)

filepath = path + 'e{epoch:02d}_val{val_loss:.6f}.h5'

tensorboard_dir = os.path.join(path, 'tensorboard')
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)

tensorboard_callback = TensorBoard(
            log_dir                = tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = batch_size,
            write_graph            = False, # True
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
)

callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-9, epsilon=0.00001, verbose=1, mode='min'),
        ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, verbose=0),
        tensorboard_callback
]

In [ ]:
history = model.fit(images_list, mass_list, validation_split=0.2, epochs = 100, callbacks = callbacks, batch_size = 64)

Epoch 1/100
3/3 [==============================] - 651s 236s/step - loss: 99.7840 - val_loss: 172.8464 - lr: 0.0010
Epoch 2/100
3/3 [==============================] - 649s 236s/step - loss: 450.0320 - val_loss: 168.0730 - lr: 0.0010
Epoch 3/100
3/3 [==============================] - 605s 214s/step - loss: 1044.5721 - val_loss: 328.5539 - lr: 0.0010
Epoch 4/100
3/3 [==============================] - 643s 233s/step - loss: 1787.6162 - val_loss: 302.4003 - lr: 0.0010
Epoch 5/100
3/3 [==============================] - 651s 237s/step - loss: 1132.7147 - val_loss: 558.1561 - lr: 0.0010
Epoch 6/100
3/3 [==============================] - 649s 234s/step - loss: 673.6477 - val_loss: 93.3570 - lr: 0.0010
Epoch 7/100
3/3 [==============================] - 605s 215s/step - loss: 411.8558 - val_loss: 106.1951 - lr: 0.0010
Epoch 8/100
3/3 [==============================] - 644s 234s/step - loss: 201.4298 - val_loss: 72.7175 - lr: 0.0010
Epoch 9/100
3/3 [==============================] - 619s 219s/ste

KeyboardInterrupt: ignored

GENERATOR DATA

In [ ]:
model.save('drive/MyDrive/CRUPA/model_save/model_LSTM_04_2020.h5')

In [ ]:
i = 225
img_sample = images_list[i]
img_sample = np.expand_dims(img_sample, axis=0)
pred = model.predict(img_sample)
print(images_info_df['mass'][i], '-true')
print(pred, '- predict')


130 -true
[[115.10689]] - predict


In [ ]:
for i in range(0,100):
  img_sample = images_list[i]
  img_sample = np.expand_dims(img_sample, axis=0)
  pred = model.predict(img_sample)
  print(images_info_df['mass'][i], '-true')
  print(pred, '- predict')

100 -true
[[93.44557]] - predict
100 -true
[[95.81205]] - predict
100 -true
[[98.02221]] - predict
100 -true
[[96.75058]] - predict
100 -true
[[99.29523]] - predict
100 -true
[[84.503624]] - predict
100 -true
[[88.53787]] - predict
100 -true
[[93.9474]] - predict
100 -true
[[106.849884]] - predict
100 -true
[[100.32175]] - predict
100 -true
[[14.584902]] - predict
100 -true
[[12.55113]] - predict
100 -true
[[12.936537]] - predict
100 -true
[[8.26472]] - predict
100 -true
[[10.540631]] - predict
100 -true
[[15.215013]] - predict
100 -true
[[10.447259]] - predict
100 -true
[[15.75032]] - predict
100 -true
[[14.94017]] - predict
100 -true
[[16.425879]] - predict
100 -true
[[94.08453]] - predict
100 -true
[[94.85288]] - predict
100 -true
[[102.68475]] - predict
100 -true
[[105.30854]] - predict
100 -true
[[102.57678]] - predict
100 -true
[[104.567986]] - predict
100 -true
[[108.55281]] - predict
100 -true
[[108.076035]] - predict
100 -true
[[106.644226]] - predict
100 -true
[[105.42126]] -

KeyboardInterrupt: ignored